## 과제
1. 웹크롤링
2. csv파일을 이용한 데이터 분석(전처리, 그래프)

두가지 합쳐서 해도 됨

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support.select import Select
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import urllib.request

#맛집 데이터 크롤링
print("=" *50)
print("    전국의 맛집 정보 추출하기 + 이미지 포함")
print("=" *50)

trial = ['','서울', '강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '세종', '울산', '인천', '전남', '전북', '제주', '충남', '충북']

busan = ['', '강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구']
category = ['', '배달', '밥집', '카페', '술집', '고깃집', '횟집', '한식', '중식', '일식', '양식', '아시안', '멕시칸', '이탈리안', '프렌치', '뷔페', '브런치', '패스트푸드','분식', '국물요리', '면요리', '해산물']

#지역 입력
for idx in range(len(trial)):
    if idx % 6 == 0:
        print('\n')
    
    if idx == 0:
        print(f" {idx}. 전체", end='\t')
        continue

    print(f" {idx}. {trial[idx]}", end='\t')
print("\n")
trial_idx = int(input('위 지역 중 자료를 수집할 지역의 번호를 적어주세요~'))
print("\n")
print("=" * 50)

#시군구 입력 (부산일 경우만 입력 받을 거임)
busan_idx = 0
if trial_idx == 9: #부산
    for idx in range(len(busan)):
        if idx % 6 == 0:
            print('\n')
    
        if idx == 0:
            print(f" {idx}. 전체", end='\t')
            continue
        print(f" {idx}. {busan[idx]}", end='\t')
    print("\n")
    busan_idx = int(input('위 지역 중 자료를 수집할 지역의 번호를 적어주세요~'))
print("\n")
print("=" * 50)

#카테고리 입력
for idx in range(len(category)):
    if idx % 6 == 0:
        print('\n')
    
    if idx == 0:
        print(f" {idx}. 전체", end='\t')
        continue

    print(f" {idx}. {category[idx]}", end='\t')

print("\n")
category_idx = int(input('위 지역 중 자료를 수집할 지역의 번호를 적어주세요~'))
print("\n")
print("=" * 50)


# file_bool = int(input('결과를 엑셀 파일에 저장할까요?  1. Yes  2. No'))

# f_name = ''
# if file_bool == 1:
#     f_dir = input('결과 파일을 저장할 폴더명만 적어주세요! ')
#     now = time.localtime()
#     s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
#     os.makedirs(f_dir + '_' +s + '_' + trial[trial_idx] + busan[busan_idx] + category[category_idx])
#     os.chdir(f_dir + '_' +s + '_' + trial[trial_idx] + busan[busan_idx] + category[category_idx])
#     f_name = trial[trial_idx] + busan[busan_idx] + '_' +category[category_idx] + '_' + s + 'csv'

# 크롬 브라우저 실행
path = "chromedriver.exe"
url = 'https://www.diningcode.com/list.dc?addr=' + trial[trial_idx] + '%20' + busan[busan_idx] +'&keyword=' + category[category_idx]
#driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()
driver.get(url)
time.sleep(4)

# 스크롤 다운
def scroll_down(driver, tag):
    #driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    driver.execute_script("arguments[0].scrollBy(0, arguments[0].scrollHeight);", tag)
    time.sleep(1)


list_selector = "#root > div > div"
ac = driver.find_element(By.CSS_SELECTOR, list_selector)
ti = driver.find_element(By.CLASS_NAME, 'Filter__Info__Title')
arrow = driver.find_element(By.ID, 'arrow')

more = driver.find_element(By.XPATH, '//*[@id="map"]/button[2]')
more.click()

while True:
    try:
        more = driver.find_element(By.XPATH, '//*[@id="map"]/button[2]')
        more.click()
    except Exception:
        break

time.sleep(3)
arrow.send_keys(Keys.END)

# driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight);", ac)


#select_date = Select(driver.find_elements_by_xpath('/html/body/div/div[2]/div[2]/div[3]/form/div/div[1]/div/select'))

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')
restaurant_list = soup.select('#root > div > div > div > a')
# print(restaurant_list)

ranking = []
location = []
rest_category =[]
score = []
user_score = []

# 이미지 저장용 폴더생성
# img_dir = f_dir+"\\images"
# os.makedirs(img_dir)
# os.chdir(img_dir)

for li in restaurant_list:
    rank = li.select_one('.RHeader > .Info > .InfoHeader > h2')
    print(rank.get_text())


#yummy = pd.DataFrame()
#yummy['점수'] = pd.Series()


    전국의 맛집 정보 추출하기 + 이미지 포함


 0. 전체	 1. 서울	 2. 강원	 3. 경기	 4. 경남	 5. 경북	

 6. 광주	 7. 대구	 8. 대전	 9. 부산	 10. 세종	 11. 울산	

 12. 인천	 13. 전남	 14. 전북	 15. 제주	 16. 충남	 17. 충북	







 0. 전체	 1. 배달	 2. 밥집	 3. 카페	 4. 술집	 5. 고깃집	

 6. 횟집	 7. 한식	 8. 중식	 9. 일식	 10. 양식	 11. 아시안	

 12. 멕시칸	 13. 이탈리안	 14. 프렌치	 15. 뷔페	 16. 브런치	 17. 패스트푸드	

 18. 분식	 19. 국물요리	 20. 면요리	 21. 해산물	



1. 현완단겸 상추튀김 상무
2. 서울곱창 광주송정역
3. 아오리따 광주동구
4. 허슬피자 광주
5. 피터테이블 수완점
6. 양림전집 광주
7. 미드픽 광주
8. 오카츠 상무지구
9. 광신보리밥 두암동
10. 황톳길 광주
11. 먼데이 오프 플리즈 광주
12. 고려인마을가족카페 광주
13. 심야식당 상무지구
14. 동화 광주동구
15. 부나비 일곡동
16. 사운드오브뮤직 광주
17. 어등 진 국밥 광주광산구
18. 와인에빠지다 광주
19. 코지코지 광주
20. 뜸 광주
21. 스페샬나잇트 광주송정역
22. 백수간재미 광주
23. Arrange 수완지구
24. NIRVANA 광주
25. 투다리 수완526점
26. 무등산생고기 식육식당 상무지구
27. 동성떡갈비 광주송정역
28. 돈일촌 충장로
29. 웜그레이 충장로
30. 비워두기 봉선동
31. 선술집 정 광주광산구
32. 정스트하우스 광주서구
33. 심야식당 진테츠 광주남구
34. 청담이상 수완점
35. 극한 수완지구
36. 하이드아웃 충장로
37. 역전할머니 광주호남대점
38. 운율 광주동구
39. 영도씨 첨단
40. 독도집 광주서구
41. 역전할머니맥주 첨단
42. 리비어밤코 상무지구
43. 돈구이불붙었네 운암동
44. 저팔계 봉선동
45. 하나비 상무지구
46. 